Downloading and parsing drugbank

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
#from drugbank_downloader import download_drugbank, open_drugbank
import repos_tools
import xml.etree.ElementTree as ET
import pandas as pd
import re

The `download_drugbank` function of `drugbank_downloader` failed with `TypeError: get_config() takes 2 positional arguments but 3 were given`.

Fortunately, another drugbank parser, written by [dhimmel](https://github.com/dhimmel), worked seamlessly; see below.

## dhimmel's parser

I took dhimmel's code from [parse.ipynb](https://github.com/dhimmel/drugbank/blob/gh-pages/parse.ipynb) and wrapped it in various functions in `repos_tools`.

In [2]:
root = repos_tools.get_drugbank_xml_root(xml_path='/home/attila/CTNS/resources/drugbank/drugbank.5.1.8.xml')

Getting drugs in a DataFrame.

In [3]:
drugs = repos_tools.get_drugbank_drugs(root)
drugs.to_csv('../../results/2021-08-11-drugbank/drugbank-all-drugs.csv')
drugs

,name,type,groups,atc_codes,categories,inchikey,inchi,description
drugbank_id,,,,,,,,
DB00001,Lepirudin,biotech,approved,[B01AE02],"[Amino Acids, Peptides, and Proteins, Anticoag...",None,None,Lepirudin is identical to natural hirudin exce...
DB00002,Cetuximab,biotech,approved,[L01XC06],"[Amino Acids, Peptides, and Proteins, Antibodi...",None,None,Cetuximab is an epidermal growth factor recept...
DB00003,Dornase alfa,biotech,approved,[R05CB13],"[Amino Acids, Peptides, and Proteins, Cough an...",None,None,Dornase alfa is a biosynthetic form of human d...
DB00004,Denileukin diftitox,biotech,approved|investigational,[L01XX29],"[ADP Ribose Transferases, Amino Acids, Peptide...",None,None,A recombinant DNA-derived cytotoxic protein co...
DB00005,Etanercept,biotech,approved|investigational,[L04AB01],"[Agents reducing cytokine levels, Amino Acids,...",None,None,Dimeric fusion protein consisting of the extra...
...,...,...,...,...,...,...,...,...
DB16420,Vibostolimab,biotech,investigational,[],[],None,None,Vibostolimab is under investigation in clinica...
DB16421,Volagidemab,biotech,investigational,[],"[Amino Acids, Peptides, and Proteins, Antibodi...",None,None,Volagidemab is under investigation in clinical...
DB16422,Quavonlimab,biotech,investigational,[],[],None,None,Quavonlimab is under investigation in clinical...


Getting proteins into a DataFrame.  The HGNC ID will be used to obtain NCBI Entrez Name.

In [4]:
proteins = repos_tools.get_drugbank_proteins(root)
proteins.to_csv('../../results/2021-08-11-drugbank/drugbank-all-proteins.csv')
proteins

category                              organism  \
drugbank_id uniprot_id                                                      
DB00001     P00734           target                                Humans   
DB00002     P00533           target                                Humans   
            O75015           target                                Humans   
            P02745           target                                Humans   
            P02746           target                                Humans   
...                             ...                                   ...   
DB15982     Q9UNQ0      transporter                                Humans   
DB16019     P07288           target                                Humans   
DB16353     P23467           target                                Humans   
DB16370     P28907           target                                Humans   
DB16385     Q05320           target  Zaire ebolavirus (strain Mayinga-76)   

                       known_action     actions  \
drugbank_id uniprot_id                            
DB00001     P00734              yes   inhibitor   
DB00002     P00533              yes  antagonist   
            O75015          unknown               
            P02745          unknown               
            P02746          unknown               
...                             ...         ...   
DB15982     Q9UNQ0          unknown   substrate   
DB16019     P07288              yes      binder   
DB16353     P23467              yes   inhibitor   
DB16370     P28907              yes   inhibitor   
DB16385     Q05320              yes  antagonist   

                                                                     name  \
drugbank_id uniprot_id                                                      
DB00001     P00734                                            Prothrombin   
DB00002     P00533                       Epidermal growth factor receptor   
            O75015      Low affinity immunoglobulin gamma Fc region re...   
            P02745                  Complement C1q subcomponent subunit A   
            P02746                  Complement C1q subcomponent subunit B   
...                                                                   ...   
DB15982     Q9UNQ0             ATP-binding cassette sub-family G member 2   
DB16019     P07288                              Prostate-specific antigen   
DB16353     P23467        Receptor-type tyrosine-protein phosphatase beta   
DB16370     P28907                                  ADP-ribosyl cyclase 1   
DB16385     Q05320                                  Envelope glycoprotein   

                          hgnc_id  
drugbank_id uniprot_id             
DB00001     P00734      HGNC:3535  
DB00002     P00533      HGNC:3236  
            O75015      HGNC:3620  
            P02745      HGNC:1241  
            P02746      HGNC:1242  
...                           ...  
DB15982     Q9UNQ0        HGNC:74  
DB16019     P07288      HGNC:6364  
DB16353     P23467      HGNC:9665  
DB16370     P28907      HGNC:1667  
DB16385     Q05320            NaN  

[27169 rows x 6 columns]

In [5]:
%connect_info

{
  "shell_port": 55075,
  "iopub_port": 38429,
  "stdin_port": 36439,
  "control_port": 56161,
  "hb_port": 53887,
  "ip": "127.0.0.1",
  "key": "f8fb79cd-6c5b85447a178006e30f8a25",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-0b3acdc4-701b-4b11-8bcf-4b8698b320b0.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
